In [1]:
import evaluate
import pandas as pd
import torch
from datasets import load_dataset
from dotenv import load_dotenv
from evaluate import evaluator
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForSequenceClassification, pipeline


In [2]:
load_dotenv()

True

## Load the model. Using pretrained model with quantization. Test for it's performance.

In [3]:
models = [
    'peft/bert-base-uncased',
    'peft/bert-large-uncased',
    'peft/roberta-base',
    'peft/roberta-large',
    'peft/distilbert-base-uncased',
]

In [5]:
data = load_dataset("imdb", split="test").shuffle(seed=42).select(range(1000))
task_evaluator = evaluator('sentiment-analysis')

In [ ]:
%%time

# Without quantization 
results = []

for model_id in models:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_id,
        num_labels=2,
    )
    eval_results = task_evaluator.compute(
        model_or_pipeline=pipeline('sentiment-analysis', model=model, tokenizer=tokenizer),
        data=data,
        label_mapping={"LABEL_0": 0, "LABEL_1": 1},
        metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    )
    print(f'{model_id=} {eval_results=}')
    results.append(eval_results)

In [0]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_weight=True,
    bnb_4bit_activation=True
)

In [ ]:
%%time

# With quantization
quantization_results = []

for model_id in models:
    model = AutoModelForSequenceClassification.from_pretrained(model_id, load_in_4bit=True)
    tokenizer = AutoTokenizer.from_pretrained(model_id)

    # model = load_and_quantize_model(model, bnb_quantization_config=bnb_config)

    eval_results = task_evaluator.compute(
        model_or_pipeline=pipeline('sentiment-analysis', model=model, tokenizer=tokenizer),
        data=data,
        label_mapping={"LABEL_0": 0, "LABEL_1": 1},
        metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    )
    print(f'{model_id=} {eval_results=}')
    quantization_results.append(eval_results)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\aksid\PycharmProjects\ReviewCategoriser\venv\lib\site-packages\bitsandbytes\nn\modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
C:\Users

model_id='peft/bert-base-uncased' eval_results={'accuracy': 0.898, 'recall': 0.9467213114754098, 'precision': 0.8587360594795539, 'f1': 0.9005847953216374, 'total_time_in_seconds': 31.590575300040655, 'samples_per_second': 31.655010727161816, 'latency_in_seconds': 0.031590575300040655}


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-large-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df = pd.DataFrame(results, index=models)
df[["accuracy", "recall", "precision", "f1", "total_time_in_seconds", 'samples_per_second', 'latency_in_seconds']]

In [ ]:
dfq = pd.DataFrame(quantization_results, index=models)
dfq[["accuracy", "recall", "precision", "f1", "total_time_in_seconds", 'samples_per_second', 'latency_in_seconds']]